In [1]:
import numpy as np

# Problem Description

It is given the following Objective Function and it's asked to minimize it. 
$$f(x,y) = x^2 + y^2$$

This function has a global minimum in (0,0), where f(0,0) = 0.

In [2]:
def objective(x):
    return (x[0]**2.0 + x[1]**2.0)

## Create Starting Population

In [3]:
def create_population(n_bits, n_population, bounds):
    population = list()
    for n in range(n_population):
        sample = np.random.randint(0, 2, n_bits*len(bounds)).tolist()
        population.append(sample)
    
    # Observation: each sample of the population is a 32-Bit-Binary String in which the First 16 Bits describe the "X" and the Last 16 Bits describe the "Y".
    return population

## Decoding Function

In [4]:
def decode(bounds, n_bits, sample):
    decoded = list()
    max_value = 2**n_bits - 1
    
    for i in range(len(bounds)):
        # Substring Extraction
        start = i * n_bits              # i=0 -> start = 0, i=1 -> start = 16
        end = (i * n_bits) + n_bits     # i=0 -> end = 16, i=1 -> end = 32 

        substring = sample[start:end]

        chars = ''.join([str(s) for s in substring])
        integer = int(chars, 2)         # Transforms the Binary Number in a Decimal Number

        value = bounds[i][0] + (integer/max_value) * (bounds[i][1] - bounds[i][0]) 
        # What happens here? If i=0 we work on the "X", else, if i=1, we work on the "Y"
            # 1) bounds[i][0] = Lower Bound for the Variable
            # 2) Mapping of the "integer" value in the "bounds" interval
        
        decoded.append(value)
    
    return decoded

## Selection

In [5]:
def selection(population, scores, k=3):
    selected_idx = np.random.randint(len(population))

    for idx in np.random.randint(0, len(population), k-1):
        if scores[idx] < scores[selected_idx]:
            selected_idx = idx
    
    return population[selected_idx]

## Crossover

In [6]:
def crossover(p1, p2, r_crossover):
    c1 = p1.copy()
    c2 = p2.copy()

    execute_crossover = np.random.rand()
    
    if execute_crossover < r_crossover:
        point = np.random.randint(1, len(p1)-2)

        c1 = p1[:point] + p2[point:]
        c2 = p2[:point] + p1[point:]
    
    return [c1, c2]

## Mutazione

In [7]:
def mutation(sample, r_mutation):
    for i in range(len(sample)):
        execute_mutation = np.random.rand()

        if execute_mutation < r_mutation:
            sample[i] = 1 - sample[i]

## Genetic Algorithm: Principal Function

In [8]:
def genetic_algorithm(objective, bounds, n_bits, n_iterations, n_population, r_crossover, r_mutation):
    # Create the Starting Population
    population = create_population(n_bits, n_population, bounds)
    
    # Initialize "Best"
    best = population[0]
    best_eval = objective(decode(bounds, n_bits, best))

    # Iterate over "n_iterations" Generations
    iteration = 1
    for gen in range(n_iterations):
        decoded_samples = list()
        for sample in population:
            decoded_samples.append(decode(bounds, n_bits, sample))
        
        scores = list()
        for sample in decoded_samples:
            scores.append(objective(sample))
        
        # (Eventually) Update the Best
        new_best_found = False
        for i in range(n_population):
            if scores[i] < best_eval:
                new_best_found = True
                best = population[i]
                best_eval = scores[i]
        
        if new_best_found == True:
            print("New \'Best\' found at Iteration %d" % iteration)
            print("New Best: " + str(best))
            print("New Best Eval: %.9f\n" % best_eval)
        
        # Create the Next Generation
        parents = list()
        children = list()

        for sample in population:
            parents.append(selection(population, scores))
        
        for i in range(0, n_population, 2):
            p1 = parents[i]
            p2 = parents[i+1]

            for c in crossover(p1, p2, r_crossover):
                mutation(c, r_mutation)
                children.append(c)
        
        population = children
        iteration += 1

# CODE'S TEST SECTION

In [9]:
# Parameters Setting
bounds = [[-5.0, 5.0], [-5.0, 5.0]]
n_iterations = 100
n_bits = 16
n_population = 100
r_crossover = 0.9
r_mutation = 1.0 / (float(n_bits) * len(bounds))

In [10]:
genetic_algorithm(objective, bounds, n_bits, n_iterations, n_population, r_crossover, r_mutation)

New 'Best' found at Iteration 1
New Best: [0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0]
New Best Eval: 0.546898165

New 'Best' found at Iteration 2
New Best: [0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0]
New Best Eval: 0.149035310

New 'Best' found at Iteration 3
New Best: [0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1]
New Best Eval: 0.034282716

New 'Best' found at Iteration 4
New Best: [0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1]
New Best Eval: 0.004052646

New 'Best' found at Iteration 5
New Best: [0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1]
New Best Eval: 0.003833313

New 'Best' found at Iteration 6
New Best: [0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0]
New Best Eval: 0.001456598

## Other Tests

In [11]:
# Creating a Starting Population

bounds = [[-5.0, 5.0], [-5.0, 5.0]]
n_pop = 10
n_bits = 16
test_population = create_population(n_pop, n_bits, bounds)

test_population

[[1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1],
 [0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0],
 [1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0],
 [0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1],
 [0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0],
 [0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1],
 [1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0],
 [0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1],
 [0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1],
 [0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1],
 [0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0],
 [1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0],
 [0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0],
 [1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0],
 [0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1],
 [0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1,

In [12]:
# Decoding Process

print("Binary Sample: " + str(test_population[0]))
decoded_sample = decode(bounds, n_bits, test_population[0])
print("Decoded Sample: " + str(decoded_sample))

Binary Sample: [1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1]
Decoded Sample: [3.7153429465171293, -4.9977111467154955]


In [13]:
# Evaluating the Population
population_with_values = dict()
for sample in test_population:
    population_with_values[str(sample)] = None
    
for sample in test_population:
    decoded_sample = decode(bounds, n_bits, sample)
    sample = str(sample)
    population_with_values[sample] = (decoded_sample)

print("Population with Values:")
population_with_values

Population with Values:


{'[1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1]': [3.7153429465171293,
  -4.9977111467154955],
 '[0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0]': [-4.193560692759594,
  -4.9978637369344625],
 '[1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0]': [1.6863508049134044,
  -5.0],
 '[0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1]': [-1.3140306706340121,
  -4.998931868467231],
 '[0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0]': [-1.4124513618677041,
  -4.999084458686198],
 '[0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1]': [-3.939497978179599,
  -4.998321507591363],
 '[1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0]': [0.869687953002213,
  -4.999084458686198],
 '[0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1]': [-0.635309376668955,
  -4.998321507591363],
 '[0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1]': [-1.683604180972,
  -4.9980163271534295],
 '[0, 1, 1, 1, 1, 1, 0, 0, 0, 0,

In [18]:
# Selection Process
scores = list()
for sample in test_population:
    decoded_sample = decode(bounds, n_bits, sample)
    scores.append(objective(decoded_sample))

parents = list()

for i in range(n_pop):
    parents.append(selection(test_population,scores))
    
parents

[[0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1],
 [1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0],
 [0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1],
 [1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0],
 [0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1],
 [0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0],
 [0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1],
 [0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1],
 [0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1],
 [1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0]]

In [22]:
# Crossover Process
p1 = parents[0]
p2 = parents[1]

c1, c2 = crossover(p1, p2, r_crossover)

print("Child 1: " + str(c1))
print("Child 2: " + str(c2))

Child 1: [0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0]
Child 2: [1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1]


In [24]:
# Mutation Process
print("Child Before Mutation: " + str(c1))
mutation(c1, r_mutation)
print("Child After Mutation: " + str(c1))

Child Before Mutation: [0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0]
Child After Mutation: [0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0]
